In [3]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
import itertools
import re
import numpy as np
sns.set_style('white')

In [5]:
# CLAMMS
def readclamms(infiles, data):
    for file in infiles:
        with open(file) as fin:
            next(fin)
            for line in fin:
                l = line.strip().split()
                chrom,x,y,sample,cnv, cn = l[0], l[1], l[2], l[4], l[5], l[6]
                if PLOT_TYPE not in sample:
                    continue
                if cnv == 'DUP':
                    cnv = 'dup'
                else:
                    cnv = 'del'
                if sample not in data:
                    data[sample] = []
                data[sample].append([chrom,int(x),int(y),cnv,int(cn), 'clamms'])

# CODEX2
def readcodex2(infiles, data):
    for file in infiles:
        with open(file) as fin:
            next(fin)
            for line in fin:
                l = line.strip().split()
                chrom,x,y,sample,cnv,cn = l[1], l[3], l[4], l[0], l[2], l[10]
                if PLOT_TYPE not in sample:
                    continue
                if cnv == 'dup':
                    pass
                if sample not in data:
                    data[sample] = []
                data[sample].append([chrom,int(x),int(y),cnv,int(cn), 'codex2'])

# CONTRA
def readcontra(infiles, data):
    for file in infiles:
        with open(file) as fin:
            next(fin)
            for line in fin:
                l = line.strip().split()
                chrom,x,y,sample,cnv = l[4], l[5], l[6], l[0], l[14]
                if PLOT_TYPE not in sample:
                    continue
                if cnv == 'gain':
                    cnv, cn = 'dup', 3
                else:
                    cnv, cn = 'del', 1
                if sample not in data:
                    data[sample] = []
                data[sample].append([chrom,int(x),int(y),cnv,cn, 'contra'])

# EXCAVATOR2
def readexcavator2(infiles, data):
    for file in infiles:
        with open(file) as fin:
            next(fin)
            for line in fin:
                l = line.strip().split()
                chrom,x,y,sample,cnv = l[1], l[2], l[3], l[0], int(l[7])
                if PLOT_TYPE not in sample:
                    continue
                if cnv >= 1:
                    cnv, cn = 'dup', 3
                elif cnv == -1:
                    cnv, cn = 'del', 1
                else:
                    cnv, cn = 'del', 0
                if sample not in data:
                    data[sample] = []
                data[sample].append([int(x),int(y),cnv,cn, 'excavator2'])

# EXOMEDEPTH
def readexomedepth(infiles, data):
    for file in infiles:
        with open(file) as fin:
            next(fin)
            for line in fin:
                l = line.strip().split()
                chrom,x,y,sample,cnv = l[7].strip('"'), l[5], l[6], l[0], l[3]
                if cnv.strip('"') == 'deletion':
                    cnv, cn = 'del', 1
                else:
                    cnv, cn = 'dup', 3
                if sample not in data:
                    data[sample] = []
                data[sample].append([chrom, int(x),int(y),cnv,cn, 'exomedepth'])

# XHMM
def readxhmm(infiles, data):
    for file in infiles:
        with open(file) as fin:
            next(fin)
            for line in fin:
                l = line.strip().split()
                chrom, xy, sample,cnv = l[4], l[2], l[0], l[1]
                _,x,y = re.split('[:-]',xy)
                if PLOT_TYPE not in sample:
                    continue
                if cnv == 'DEL':
                    cnv, cn = 'del', 1
                else:
                    cnv, cn = 'dup', 3
                if sample not in data:
                    data[sample] = []
                data[sample].append([chrom, int(x),int(y),cnv,cn, 'xhmm'])

In [1]:
# Reading GFF3 file
# Currently only reading entries for genes, ignoring alternative isoforms
infile = '/home/harald/projects/ensembl/Homo_sapiens.GRCh38.95.chr.gff3'
genesdb = {}
exonsdb = {}
with open(infile, 'r') as fin:
    for line in fin:
        if line.startswith('#'):
            continue
        l = line.strip().split()
        if 'RNA' in l[2]:
            for ann in l[8].split(';'):
                key, value = ann.split('=')
                if key == 'Name':
                    name = value
                elif key == 'ID':
                    _, ts = value.split(':')
            genesdb[ts] = name
        elif l[2] == 'exon':
            for ann in l[8].split(';'):
                key, value = ann.split('=')
                if key == 'Name':
                    name = value
                elif key == 'Parent':
                    _, ts = value.split(':')
            try:
                exonsdb[name] = ['chr'+l[0], int(l[3]), int(l[4]),ts,genesdb[ts]]
            except KeyError:
                continue
# exonsdb = {'ENSE00002234944': ['chr1', 11869, 12227, 'ENST00000456328', 'DDX11L1-202'], ...

In [6]:
# Reading collected results
methods = {'codex2':1,'exomedepth':2,'xhmm':3}
data = {}
PLOT_TYPE = 'case'
exomedepth_files = ['/home/harald/projects/somatic/test_samples/wes_cnvs/final_report_exomedepth.txt']
excavator2_files = []
codex2_files = ['/home/harald/projects/somatic/test_samples/wes_cnvs/final_report_codex2.txt']
contra_files = []
xhmm_files = ['/home/harald/projects/somatic/test_samples/wes_cnvs/final_report_xhmm.txt']
clamms_files = []
if len(exomedepth_files) > 0:
    readexomedepth(exomedepth_files, data)
if len(excavator2_files) > 0:
    readexcavator2(excavator2_files, data)
if len(contra_files) > 0:
    readcontra(contra_files, data)
if len(codex2_files) > 0:
    readcodex2(codex2_files, data)
if len(xhmm_files) > 0:
    readxhmm(xhmm_files, data)
if len(clamms_files) > 0:
    readclamms(clamms_files, data)
outfile = 'results2/cnv_{}.png'.format(PLOT_TYPE)

In [9]:
del_results = {}
dup_results = {}
for exon in exonsdb:
    chrom, start, stop, ts, gene = exonsdb[exon]
    for sample, records in data.items():
        if sample not in del_results:
            del_results[sample] = {}
            dup_results[sample] = {}
        for record in records:
            #record : [chrom, int(x),int(y),cnv=['dup','del'],cn, 'xhmm']
            if record[0] != chrom:
                continue
            if start < record[2] and record[1] < stop:
                if record[3] == 'dup':
                    dup_results[sample][exon] = dup_results[sample].get(exon, 0) + 1
                else:
                    del_results[sample][exon] = del_results[sample].get(exon, 0) + 1

In [11]:
with open('BTN-reportfile_duplicates_wes-cnv.txt', 'w') as fout:
    fout.write('sample\tgene\ttranscript\texon\tchrom\tstart\tstop\tcount\n')
    for sample, result in dup_results.items():
        for exon in result:
            chrom, start, stop, ts, gene = exonsdb[exon]
            if chrom in ['chrY','chrX']:
                continue
            if result[exon] < 2:
                continue
            fout.write('{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n'.format(sample,gene,ts,exon,chrom,start,stop,result[exon]))
with open('BTN-reportfile_deletions_wes-cnv.txt', 'w') as fout:
    fout.write('sample\tgene\ttranscript\texon\tchrom\tstart\tstop\tcount\n')
    for sample, result in del_results.items():
        for exon in result:
            chrom, start, stop, ts, gene = exonsdb[exon]
            if chrom in ['chrY','chrX']:
                continue
            if result[exon] < 2:
                continue
            fout.write('{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n'.format(sample,gene,ts,exon,chrom,start,stop,result[exon]))